In [1]:
import pandas as pd 
import duckdb 

In [2]:
conn = duckdb.connect()

In [3]:
prefix = "data/input/mbsf_medpar_denom_legacy/inpatient"
years = list(range(2000, 2016+1))
years.remove(2006)
results = []

for year in years:
    print(year)
    conn.execute(f"""
    CREATE OR REPLACE TABLE adm_data AS
    SELECT * FROM read_parquet('{prefix}_{year}.parquet');
    """)

    # Query to find duplicates
    query = f"""
    SELECT COUNT(*) AS duplicate_count
    FROM (
        SELECT adm_id, bene_id, admission_date, diagnoses, discharge_date, year, adm_source,
               COUNT(*) AS cnt
        FROM adm_data
        GROUP BY ALL
        HAVING COUNT(*) > 1
    ) AS duplicates
    """

    duplicate_count = conn.execute(query).fetchone()[0]
    results.append((year, duplicate_count))

    conn.execute("DROP TABLE adm_data;")

df_results = pd.DataFrame(results, columns=["Year", "Duplicate_Count"])

# results
print("Year-wise Duplicate Counts:")
print(df_results)

conn.close()

2000
2001
2002
2003
2004
2005
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
Year-wise Duplicate Counts:
    Year  Duplicate_Count
0   2000                0
1   2001                0
2   2002                0
3   2003                0
4   2004                0
5   2005                0
6   2007                0
7   2008                0
8   2009                0
9   2010                0
10  2011                0
11  2012                0
12  2013                0
13  2014                0
14  2015                0
15  2016                0
